In [ ]:
!pip install datasets
!pip install transformers

In [ ]:
!pip install accelerate -U
!pip install transformers[torch]

In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
!pip install evaluate

In [ ]:
! pip install datasets transformers[sentencepiece] sacrebleu

In [ ]:
from datasets import load_dataset, DatasetDict, load_metric

In [ ]:
raw_datasets = load_dataset('bible_para', 'en-es')
metric = load_metric("sacrebleu")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
<ipython-input-7-d9be9ce364f0>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository c

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 62191
    })
})

In [ ]:
train_devtest = raw_datasets['train'].train_test_split(shuffle = True, seed = 200, test_size=0.1)
posts_dev_test = train_devtest['test'].train_test_split(shuffle = True, seed = 200, test_size=0.50)
raw_datasets = DatasetDict({
    'train': train_devtest['train'],
    'validation': posts_dev_test['test'],
    'test': posts_dev_test['train']})

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 55971
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 3110
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 3110
    })
})

In [ ]:
raw_datasets["train"][172]["translation"]

{'en': 'Now this is the commandment, the statutes, and the ordinances, which Yahweh your God commanded to teach you, that you might do them in the land where you go over to possess it;',
 'es': '"Éstos, pues, son los mandamientos, las leyes y los decretos que Jehovah vuestro Dios ha mandado que os enseñara, para que los pongáis por obra en la tierra a la cual pasáis para tomarla en posesión'}

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-es"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "es"

def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
raw_datasets['train'][:2]

{'id': ['36114', '41389'],
 'translation': [{'en': 'And lest thou lift up thine eyes unto heaven, and when thou seest the sun, and the moon, and the stars, even all the host of heaven, shouldest be driven to worship them, and serve them, which the LORD thy God hath divided unto all nations under the whole heaven.',
   'es': 'No sea que al alzar tus ojos al cielo y al ver el sol, la luna y las estrellas, es decir, todo el ejército del cielo, seas desviado a postrarte ante ellos y a rendir culto a cosas que Jehovah tu Dios ha asignado a todos los pueblos de debajo del cielo'},
  {'en': 'And when Hadad was dead, Samlah of Masrekah reigned in his stead.',
   'es': 'Murió Hadad, y reinó en su lugar Samla, de Masreca'}]}

In [ ]:
preprocess_function(raw_datasets['train'][:2])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[188, 36715, 7054, 11204, 141, 33794, 3382, 4088, 7542, 2, 10, 208, 7054, 333, 1155, 5, 5336, 2, 10, 5, 14060, 2, 10, 5, 9894, 2, 455, 81, 5, 4800, 7, 7542, 2, 130, 2875, 44, 14907, 13, 9404, 167, 2, 10, 3628, 167, 2, 80, 5, 5403, 7293, 386, 11769, 9117, 4088, 81, 5500, 214, 5, 1236, 7542, 3, 0], [188, 208, 15420, 1954, 59, 2658, 2, 4517, 20413, 7, 4994, 100, 1774, 702, 42339, 16, 119, 25, 30137, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[71, 383, 15, 47, 47, 6843, 1209, 3195, 47, 4756, 11, 47, 555, 14, 3805, 2, 6, 13665, 11, 18, 6117, 2, 43, 619, 2, 163, 14, 5691, 19, 4756, 2, 9018, 48686, 8, 1444, 514, 6746, 822, 528, 11, 8, 15610, 16809, 8, 682, 15, 3649, 213, 392, 87, 13812, 8, 154, 17, 2145, 4, 5446, 19, 4756, 0], [37399, 1

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
batch_size = 32
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# Antes del fine-tuning
before_finetuning_metrics = trainer.evaluate(tokenized_datasets["test"])
print("Metrics before fine-tuning:", before_finetuning_metrics)

Metrics before fine-tuning: {'eval_loss': 1.3184878826141357, 'eval_bleu': 31.9231, 'eval_gen_len': 31.7569, 'eval_runtime': 204.2445, 'eval_samples_per_second': 15.227, 'eval_steps_per_second': 0.48}


In [ ]:
import torch
import random

# Definir el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Obtener un índice aleatorio del conjunto de datos de prueba
example = tokenized_datasets["test"][0]

# Obtener la frase de origen y su traducción de referencia
source_sentence = example["translation"]["en"]
target_translation_reference = example["translation"]["es"]

# Convertir la entrada en tensores y moverlos al dispositivo
input_ids_tensor = torch.tensor(example["input_ids"]).unsqueeze(0).to(device)  # Añadir dimensión y mover a GPU
attention_mask_tensor = torch.tensor(example["attention_mask"]).unsqueeze(0).to(device)  # Añadir dimensión y mover a GPU

# Generar la traducción usando el modelo
translated_text = model.generate(input_ids=input_ids_tensor, attention_mask=attention_mask_tensor, max_length=max_target_length)[0]

# Decodificar la traducción generada
generated_translation = tokenizer.decode(translated_text, skip_special_tokens=True, clean_up_tokenization_spaces=True)

# Imprimir los resultados antes del fine-tuning
print("Antes del fine-tuning:")
print("Frase de origen (en):", source_sentence)
print("Traducción de referencia (es):", target_translation_reference)
print("Traducción generada por el modelo (es):", generated_translation)

Antes del fine-tuning:
Frase de origen (en): Beloved, when I gave all diligence to write unto you of the common salvation, it was needful for me to write unto you, and exhort you that ye should earnestly contend for the faith which was once delivered unto the saints.
Traducción de referencia (es): Amados, mientras me esforzaba por escribiros acerca de nuestra común salvación, me ha sido necesario escribir para exhortaros a que contendáis eficazmente por la fe que fue entregada una vez a los santos
Traducción generada por el modelo (es): Amados, cuando les di toda diligencia para escribirles de la salvación común, fue necesario que yo les escribiera, y les exhorto a que contiendan seriamente por la fe que una vez fue entregada a los santos.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.087600,0.983086,42.760600,30.231500
2,0.961700,0.931718,44.309900,30.427700
3,0.904200,0.918869,44.745500,30.453400


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=5250, training_loss=1.0016419270833334, metrics={'train_runtime': 1563.2823, 'train_samples_per_second': 107.411, 'train_steps_per_second': 3.358, 'total_flos': 2944433428955136.0, 'train_loss': 1.0016419270833334, 'epoch': 3.0})

In [ ]:
after_finetuning_metrics = trainer.evaluate(tokenized_datasets["test"])
print("Metrics after fine-tuning:", after_finetuning_metrics)

Metrics after fine-tuning: {'eval_loss': 0.9167386889457703, 'eval_bleu': 44.6999, 'eval_gen_len': 30.4884, 'eval_runtime': 197.7346, 'eval_samples_per_second': 15.728, 'eval_steps_per_second': 0.496, 'epoch': 3.0}


In [ ]:
import torch
import random

# Definir el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Obtener un índice aleatorio del conjunto de datos de prueba
example = tokenized_datasets["test"][0]

# Obtener la frase de origen y su traducción de referencia
source_sentence = example["translation"]["en"]
target_translation_reference = example["translation"]["es"]

# Convertir la entrada en tensores y moverlos al dispositivo
input_ids_tensor = torch.tensor(example["input_ids"]).unsqueeze(0).to(device)  # Añadir dimensión y mover a GPU
attention_mask_tensor = torch.tensor(example["attention_mask"]).unsqueeze(0).to(device)  # Añadir dimensión y mover a GPU

# Generar la traducción usando el modelo
translated_text = model.generate(input_ids=input_ids_tensor, attention_mask=attention_mask_tensor, max_length=max_target_length)[0]

# Decodificar la traducción generada
generated_translation = tokenizer.decode(translated_text, skip_special_tokens=True, clean_up_tokenization_spaces=True)

# Imprimir los resultados después del fine-tuning
print("Después del fine-tuning:")
print("Frase de origen (en):", source_sentence)
print("Traducción de referencia (es):", target_translation_reference)
print("Traducción generada por el modelo (es):", generated_translation)

Después del fine-tuning:
Frase de origen (en): Beloved, when I gave all diligence to write unto you of the common salvation, it was needful for me to write unto you, and exhort you that ye should earnestly contend for the faith which was once delivered unto the saints.
Traducción de referencia (es): Amados, mientras me esforzaba por escribiros acerca de nuestra común salvación, me ha sido necesario escribir para exhortaros a que contendáis eficazmente por la fe que fue entregada una vez a los santos
Traducción generada por el modelo (es): Amados, cuando me esforcé por escribiros acerca de la salvación común, me fue necesario escribiros y exhortaros a que contendáis con insistencia por la fe que una vez fue entregada a los santos
